<a href="https://colab.research.google.com/github/cedro3/others/blob/master/DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## セットアップ
ライブラリーの読み込み、クラスと関数の定義

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import IPython.display as display
import PIL.Image
from tensorflow.keras.preprocessing import image

# Input image
def input(image, max_dim=None):
  img = PIL.Image.open(image)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Normalize an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

# Display an image
def show(img):
  display.display(PIL.Image.fromarray(np.array(img)))

# Calc loss
def calc_loss(img, model):
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  return  tf.reduce_sum(losses)

# Class DeepDream
class DeepDream(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),
        tf.TensorSpec(shape=[], dtype=tf.float32),)
  )
  def __call__(self, img, steps, step_size):
      loss = tf.constant(0.0)
      for n in tf.range(steps):
        with tf.GradientTape() as tape:
          tape.watch(img)
          loss = calc_loss(img, self.model)

        gradients = tape.gradient(loss, img)
        gradients /= tf.math.reduce_std(gradients) + 1e-8 
        
        img = img + gradients*step_size
        img = tf.clip_by_value(img, -1, 1)

      return loss, img

# run_simple
def run_deep_dream_simple(img, steps=100, step_size=0.01):
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  img = tf.convert_to_tensor(img)
  step_size = tf.convert_to_tensor(step_size)
  steps_remaining = steps
  step = 0
  while steps_remaining:
    if steps_remaining>100:
      run_steps = tf.constant(100)
    else:
      run_steps = tf.constant(steps_remaining)
    steps_remaining -= run_steps
    step += run_steps

    loss, img = deepdream(img, run_steps, tf.constant(step_size))

  result = deprocess(img)  
  return result

# run_octave
def octave(original_img):
   OCTAVE_SCALE = 1.30
   img = tf.constant(np.array(original_img))
   base_shape = tf.shape(img)[:-1]
   float_base_shape = tf.cast(base_shape, tf.float32)

   for n in range(-2, 3):
        new_shape = tf.cast(float_base_shape*(OCTAVE_SCALE**n), tf.int32)
        img = tf.image.resize(img, new_shape).numpy()
        img = run_deep_dream_simple(img=img, steps=50, step_size=0.01)
        img = tf.image.resize(img, base_shape)       
        img = tf.image.convert_image_dtype(img/255.0, dtype=tf.uint8)
   return img

# サンプルデータのダウンロード

In [ ]:
!git clone https://github.com/cedro3/Sample.git

## モデルの作成

In [ ]:
# ベースモデル InceptionV3 のダウンロード
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

# Maximize the activations of these layers
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

# make model
deepdream = DeepDream(dream_model)

# octave バージョン

In [ ]:
# 静止画をDeepDreamに変換(octave)
original_img = input('./Sample/animal_pic/dog.png')
img = octave(original_img)
show(original_img)
show(img)

In [ ]:
# ビデオを静止画に変換
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './Sample/video/elephant.mp4',   # ビデオの指定
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 6
    length = 300  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
# 静止画をDeepDream画像へ変換(octave)
import glob
from tqdm import tqdm

files=[]
for name in sorted(glob.glob('./images/*.png')):
     files.append(name)

for file in tqdm(files):
     original_img=input(file)
     dream_img = octave(original_img)
     PIL.Image.fromarray(np.array(dream_img)).save(file)  

In [ ]:
# DeepDream画像をmp4に変換
!ffmpeg -r 6 -i images/%06d.png -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# simple バージョン

In [ ]:
# 静止画をDeepDreamに変換(simple)
original_img = input('./Sample/animal_pic/dog.png')
img = run_deep_dream_simple(original_img)
show(original_img)
show(img)

In [ ]:
# ビデオを静止画に変換
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './Sample/video/elephant.mp4',   # ビデオの指定
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 6
    length = 300  # 最大フレーム数
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

In [ ]:
# 静止画をDeepDream画像へ変換(simple)
import glob
from tqdm import tqdm

files=[]
for name in sorted(glob.glob('./images/*.png')):
     files.append(name)

for file in tqdm(files):
     original_img=input(file)
     dream_img = run_deep_dream_simple(img=original_img, steps=100, step_size=0.01)
     PIL.Image.fromarray(np.array(dream_img)).save(file)  

In [ ]:
# DeepDream画像をmp4に変換
!ffmpeg -r 6 -i images/%06d.png -vcodec libx264 -pix_fmt yuv420p output2.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output2.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")